# Cython

http://docs.cython.org/en/latest/src/quickstart/cythonize.html

In [18]:
def py_f(x):
    return x**2-x

def py_integrate_f(a, b, N):
    s = 0
    dx = (b-a)/N
    for i in range(N):
        s += py_f(a+i*dx)
    return s * dx

In [21]:
%time a = [py_integrate_f(0, 100, 1000) for i in range(10000)]

CPU times: user 2.96 s, sys: 13.1 ms, total: 2.97 s
Wall time: 3.03 s


In [ ]:
%load_ext Cython

In [112]:
%%cython --annotate

def f(x):
    return x**2-x

def integrate_f(a, b, N):
    s = 0
    dx = (b-a)/N
    for i in range(N):
        s += f(a+i*dx)
    return s * dx

In [23]:
%time a = [integrate_f(0, 100, 1000) for i in range(10000)]

CPU times: user 1.95 s, sys: 11.4 ms, total: 1.96 s
Wall time: 1.99 s


## Speedup around 35% for cython compiling

In [113]:
%%cython --annotate

def c_f(double x):
    return x**2-x

def c_integrate_f(double a, double b, int N):
    cdef int i
    cdef double s, dx
    s = 0
    dx = (b-a)/N
    for i in range(N):
        s += c_f(a+i*dx)
    return s * dx

In [28]:
%time a = [c_integrate_f(0, 100, 1000) for i in range(10000)]

CPU times: user 522 ms, sys: 2.93 ms, total: 525 ms
Wall time: 527 ms


## Speedup 4x using static typing variable

* Since the iterator variable i is typed with C semantics, the for-loop will be compiled to pure C code. Typing a, s and dx is important as they are involved in arithmetic within the for-loop; typing b and N makes less of a difference, but in this case it is not much extra work to be consistent and type the entire function.

In [114]:
%%cython --annotate

cdef double cc_f(double x) except? -2:
    return x**2-x

def cc_integrate_f(double a, double b, int N):
    cdef int i
    cdef double s, dx
    s = 0
    dx = (b-a)/N
    for i in range(N):
        s += cc_f(a+i*dx)
    return s * dx

In [104]:
%time a = [cc_integrate_f(0, 100, 1000) for i in range(10000)]

CPU times: user 18.3 ms, sys: 1.08 ms, total: 19.3 ms
Wall time: 18.6 ms


## Speedup 150x using typing functions

In [111]:
%%cython --annotate

cdef double a_f(double x):
    return x**2-x

def a_integrate_f(double a, double b, int N):
    cdef int i
    cdef double s, dx
    s = 0
    dx = (b-a)/ N
    for i in range(N):
        s += a_f(a+i*dx)
    return s * dx

## how to know which part to use typing
* Lines are colored according to the level of “typedness” – white lines translate to pure C, while lines that require the Python C-API are yellow (darker as they translate to more C-API interaction). Lines that translate to C code have a plus (+) in front and can be clicked to show the generated code.